<a href="https://colab.research.google.com/github/pakmingc/Instagram/blob/main/instagram_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install instaloader

import os
import instaloader
import time
import requests
from google.colab import drive

def download_instagram_content():
    # Mount Google Drive
    drive.mount('/content/drive')

    # Initialize Instaloader
    L = instaloader.Instaloader(download_comments=False, save_metadata=False, compress_json=False,
                                dirname_pattern='/content/drive/My Drive/instagram_downloads/{target}')

    # Define your credentials
    username = input("Please enter your Instagram username: ")
    password = input("Please enter your Instagram password: ")

    max_retries = 3
    retry_delay = 5

    while True:
        try:
            # Login
            L.context.log("Logging in...")
            L.context.login(username, password)
            L.context.log("Login successful.")
            break
        except instaloader.exceptions.TwoFactorAuthRequiredException:
            # Handle two-factor authentication
            two_factor_code = input("Please enter your two-factor authentication code: ")
            L.context.two_factor_login(two_factor_code)
            L.context.log("Login successful with two-factor authentication.")
            break
        except instaloader.exceptions.BadCredentialsException:
            print("Invalid username or password. Please try again.")
            return
        except instaloader.exceptions.ConnectionException as e:
            print(f"Connection error occurred: {str(e)}")
            retry = input("Do you want to retry the login? (y/n): ").lower()
            if retry == 'y':
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print("Login aborted.")
                return
        except instaloader.exceptions.InvalidArgumentException:
            print("Invalid argument provided. Please check your input and try again.")
            return
        except Exception as e:
            print(f"An error occurred during login: {str(e)}")
            return

    # Ask the user for the Instagram username they want to download content from
    target_username = input("Please enter the Instagram username to download content from: ")
    print("Using username:", target_username)

    if "@" in target_username or " " in target_username:
        print("Invalid format, please re-enter.")
        target_username = input("Please enter the Instagram username: ")

    if not target_username:
        print("Username cannot be empty.")
        exit()

    # Provide options for download type
    print("Download type: A>Photos B>Videos C>Text D>All")
    download_type = input("Please select download type: ").upper()

    download_pictures = 'A' in download_type
    download_videos = 'B' in download_type
    download_text = 'C' in download_type
    download_all = 'D' in download_type

    # Get the user's posts
    profile = instaloader.Profile.from_username(L.context, target_username)

    # Ask the user how many items they want to download
    num_items = input("How many recent posts do you want to download? (leave blank for all): ")
    num_items = int(num_items) if num_items.isdigit() else None

    downloaded_items = 0

    # Iterate through each post
    for post in profile.get_posts():
        if num_items is not None and downloaded_items >= num_items:
            break

        # Check the post type and process according to the user's download preference
        if post.typename == 'GraphSidecar' or (post.is_video and (download_videos or download_all)) or (not post.is_video and (download_pictures or download_all)):
            # Download the post using instaloader's download_post function
            L.download_post(post, target=target_username)
            downloaded_items += 1

        # If the user wants to download text, check if the post contains text and download it
        if post.caption and (download_text or download_all):
            text_content = post.caption
            text_file_path = os.path.join(f"/content/drive/My Drive/instagram_downloads/{target_username}", f"{post.date_local}_text.txt")
            with open(text_file_path, "w", encoding='utf-8') as text_file:
                text_file.write(text_content)
            if download_text:
                downloaded_items += 1

    print(f"Downloaded {downloaded_items} items.")

    # Provide the download link
    download_link = f"/content/drive/My Drive/instagram_downloads/{target_username}"
    print(f"Download link: {download_link}")

# Start the download process
if __name__ == "__main__":
    try:
        download_instagram_content()
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

print("Done!")